In [158]:
!nvidia-smi

Mon Nov  2 20:20:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [159]:
import torch
from transformers import AutoTokenizer, AutoModel

In [160]:
def dataset(filename):
    with open(filename,'r') as fp:
        lines = [line.strip() for line in fp]
    return lines

In [161]:
def wordData(data):
    wordLines = data
    words = []
    probabilities = []
    wordList = []
    pos = []
    empty = []
    for line in wordLines:
        lineSplit = line.strip().split('\t')
        if line:
            word = lineSplit[1]
            prob = lineSplit[4]
            temp = lineSplit[5]
            words.append(word)
            probabilities.append(prob)
            pos.append(temp)
        elif not (len(empty) and []):
            wordList.append((words, pos, probabilities))
            words = []
            probabilities = []
            pos = []
    return wordList

In [162]:
def wordtest(data):
    wordLines = data
    words = []
    testWord = []
    empty = []
    for line in wordLines:
        lineSplit = line.strip().split('\t')
        if line:
            word = lineSplit[1]            
            words.append(word)
        elif not len(empty):
            testWord.append(words)
            words = []       
    return testWord

In [163]:
def preTokenizing(data):
    t = []
    po = []
    prob = []
    for i,j,k in data:
        text = []
        pos = []
        probs = []
        for l in i:
            text.append(l)
        for m in j:
            pos.append(m)
        for n in k:
            probs.append(n)
        t.append(text)
        po.append(pos)
        prob.append(probs)
    return t,po, prob

In [164]:
TRAINING_FILE = "train.txt"
DEV_FILE = "dev.txt"
TEST_FILE = "test_data.txt"

tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-en")


In [165]:
trainText = wordData(dataset(TRAINING_FILE))
testEval = wordtest(dataset(TEST_FILE))
devText = wordData(dataset(DEV_FILE))


trainWords,trainTags, trainLabels = preTokenizing(trainText)
devWords, devTags, devLabels = preTokenizing(devText)

tokenized_text = []
for i in trainWords:
    sent = ""
    for j in i:
        if sent == "":
            sent += j
        else:
            sent = sent + " " + j
    tokens = tokenizer.tokenize(sent)
    tokenized_text.append(tokens)
    

# Encoding each word of the sentence and appending to a list
en=[]
for i in tokenized_text:
  en1 = []
  for j in i:
    token_ids = en1.append(tokenizer.encode_plus(j, add_special_tokens=False, return_attention_mask=False, return_tensors='pt'))
  en.append(en1)
  en1 = []
# Getting the input id's only for the list and passing them to the model 
words = []  
for d in en:
    words2 = []
    for i in d:
        words2.append(i['input_ids'])
    words.append(words2)
    words2 = []     

In [166]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained('nghuyong/ernie-2.0-en')
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens):
        _, pooled_output = self.bert(tokens)
        linear_output = self.linear(pooled_output)
        proba = self.sigmoid(linear_output)
        proba = proba.data.tolist()
        return proba

In [167]:
bert_clf = BertBinaryClassifier()
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=3e-6)
bert_clf.train()
token_answers = []
for epoch_num in range(1):
    for batch_data in words:
        answers_temp = []
        for i in batch_data:
            probas = bert_clf(i)
            for item in probas:
                for i in item:
                    answers_temp.append(i)            
        token_answers.append(answers_temp)
        print(answers_temp)
        answers_temp = []
        bert_clf.zero_grad()
        optimizer.step()
        break
      

[0.4695046544075012, 0.5315011739730835, 0.5227498412132263]
